# Loading data into astir

## 0. Loading necessary libraries

In [1]:
# !pip install -e ../../..
from astir.data_readers import from_csv_yaml
import pandas as pd
%load_ext autoreload
%autoreload 2

## 1. Starting Astir within python

## 2. Loading from csv and yaml files

## 3. Loading from a directory of csvs and yaml

## 4. Loading from loom

## 5. Loading from anndata

We can read in data from the [AnnData](https://anndata.readthedocs.io/en/stable/anndata.AnnData.html) format, along with a `yaml` file containing marker information using the `from_anndata_yaml` function:

In [54]:
from astir.data_readers import from_anndata_yaml
ast = from_anndata_yaml(anndata_file="../../../astir/tests/test-data/adata_small.h5ad", 
                        marker_yaml="../../../astir/tests/test-data/jackson-2020-markers.yml",
                        protein_name="protein",
                        cell_name="cell_name",
                        batch_name="batch")
print(ast)

Astir object with 6 cell types, 4 cell states, and 10 cells.


Some notes:

1. The protein and cell names are taken from `adata.var[protein_name]` and `adata.obs[cell_name]` respectively if specified, and `adata.var_names` and `adata.obs_names` otherwise.

2. If `batch_name` is sepecified, the corresponding column of `adata.var` will be assumed as the batch variable and turned into a design matrix.

In [64]:
ast.get_type_dataset().get_features()

['CD20',
 'CD3',
 'CD45',
 'CD68',
 'Cytokeratin 14',
 'Cytokeratin 19',
 'Cytokeratin 5',
 'Cytokeratin 7',
 'Cytokeratin 8/18',
 'E-Cadherin',
 'Fibronectin',
 'Her2',
 'Vimentin',
 'pan Cytokeratin']